# Multiprocessing

In [15]:
from multiprocessing import Process, Value, Array, Lock
import os
import time

In [23]:
def square_numbers():
    for i in range(1000):
        i*i

if __name__ == "__main__":
    processes = []
    num_processes = os.cpu_count()
    # number of CPUs on the machine. Usually a good choice for the number of processes
    
    # create processes and asign a function for each process
    for i in range(num_processes):
        process = Process(target=square_numbers)
        processes.append(process)
        
    # start all processes
    for process in processes:
        process.start()
        
    # wait for all processes to finish and block the main program until these processes are finished
    for process in processes:
        process.join()

### Sharing single value

In [9]:
if __name__ == "__main__":
    shared_number = Value('i',0)
    print('Number at beginning is', shared_number.value)

Number at beginning is 0


In [24]:
def add_100(number):
    for i in range(100):
        time.sleep(0.01)
        number.value += 1

if __name__ == "__main__":
    shared_number = Value('i',0)
    print('Number at beginning is', shared_number.value)
    
    p1 = Process(target=add_100,args=(shared_number,))
    p2 = Process(target=add_100,args=(shared_number,))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print('number at the end is', shared_number.value)
# we may get not 200 value because 2 processes can get access to the same variable in the same time

Number at beginning is 0
number at the end is 200


In [25]:
def add_100(number, lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        number.value += 1
        lock.release()
        
if __name__ == "__main__":
    lock = Lock()
    shared_number = Value('i',0)
    print('Number at beginning is', shared_number.value)
    
    p1 = Process(target=add_100,args=(shared_number,lock))
    p2 = Process(target=add_100,args=(shared_number,lock))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print('number at the end is', shared_number.value)

Number at beginning is 0
number at the end is 200


In [26]:
# using lock as a context manager
def add_100(number, lock):
    for i in range(100):
        time.sleep(0.01)
        with lock:
            number.value += 1
        
if __name__ == "__main__":
    lock = Lock()
    shared_number = Value('i',0)
    print('Number at beginning is', shared_number.value)
    
    p1 = Process(target=add_100,args=(shared_number,lock))
    p2 = Process(target=add_100,args=(shared_number,lock))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print('number at the end is', shared_number.value)

Number at beginning is 0
number at the end is 200


### Sharing array

In [42]:
def add_100(numbers, lock):
    for i in range(100):
        time.sleep(0.01)
        for i in range(len(numbers)):
            numbers[i] += 1
        
if __name__ == "__main__":
    lock = Lock()
    shared_array = Array('d', [0.0, 100.0, 200.0])
    print('Array at beginning is', shared_array[:])
    
    p1 = Process(target=add_100,args=(shared_array,lock))
    p2 = Process(target=add_100,args=(shared_array,lock))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print('Array at the end is', shared_array[:])
# we have a raise condition here so we have to use lock

Array at beginning is [0.0, 100.0, 200.0]
Array at the end is [200.0, 300.0, 400.0]


In [43]:
def add_100(numbers, lock):
    for i in range(100):
        time.sleep(0.01)
        for i in range(len(numbers)):
            with lock:
                numbers[i] += 1
        
if __name__ == "__main__":
    lock = Lock()
    shared_array = Array('d', [0.0, 100.0, 200.0])
    print('Array at beginning is', shared_array[:])
    
    p1 = Process(target=add_100,args=(shared_array,lock))
    p2 = Process(target=add_100,args=(shared_array,lock))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print('Array at the end is', shared_array[:])

Array at beginning is [0.0, 100.0, 200.0]
Array at the end is [200.0, 300.0, 400.0]


In [44]:
from multiprocessing import Queue

In [45]:
def square(numbers, queue):
    for i in numbers:
        queue.put(i*i)

def make_negative(numbers, queue):
    for i in numbers:
        queue.put(-1*i)
    
if __name__ == "__main__":
    numbers = range(1, 6)
    q = Queue()
    
    p1 = Process(target=square, args=(numbers, q))
    p2 = Process(target=make_negative, args=(numbers, q))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    while not q.empty():
        print(q.get()) # q.get() returns and removes first element in queue

1
4
9
16
25
-1
-2
-3
-4
-5


In [46]:
from multiprocessing import Pool

In [47]:
def cube(number):
    return number * number * number

if __name__ == "__main__":
    numbers = range(10)
    pool = Pool()
    # map, apply, join, close the most important methods from Pool
    result = pool.map(cube, numbers)
    
    pool.close()
    pool.join()
    print(result)
    

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]


In [57]:
def cube(number):
    return number

if __name__ == "__main__":
    numbers = range(10)
    pool = Pool()
    
    result = pool.map(cube, numbers)
    # pool.apply(cube, numbers[1]) # if we want one function executed by pool
    
    pool.close()
    pool.join()
    
    print(result)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
